# Read Source table


In [0]:
%sql
SELECT * FROM workspace.bronze.crm_sales_details LIMIT 5

# Drop the target table if already exists

In [0]:
%sql
drop table if exists workspace.silver.crm_sales

# Create the target table

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.silver.crm_sales 
(order_number STRING,
product_number STRING,
customer_number INT,
order_date DATE,
ship_date DATE,
due_date DATE,
sales_amount INT,
quantity INT,
price INT) USING DELTA 

# Do the Transformations in the original table & insert to the target table

### Cast the date_columns to date, do the corrections in sales & price column

In [0]:
%sql
INSERT OVERWRITE workspace.silver.crm_sales
SELECT
  sls_ord_num,
  sls_prd_key,
  sls_cust_id,
  DATE_FROM_UNIX_DATE(UNIX_DATE(CASE 
    WHEN len(sls_order_dt) != 8 OR sls_order_dt = 0 THEN NULL
    ELSE TO_DATE(CAST(sls_order_dt AS STRING), 'yyyyMMdd')
    END)) AS order_date,
  DATE_FROM_UNIX_DATE(UNIX_DATE(CASE 
    WHEN len(sls_ship_dt) != 8 OR sls_ship_dt = 0 THEN NULL  
    ELSE TO_DATE(CAST(sls_ship_dt AS STRING), 'yyyyMMdd') 
  END)) AS ship_date,
  DATE_FROM_UNIX_DATE(UNIX_DATE(CASE
    WHEN len(sls_due_dt) != 8 OR sls_due_dt = 0 THEN NULL  
    ELSE TO_DATE(CAST(sls_due_dt AS STRING), 'yyyyMMdd') 
    END)) AS due_date,
  CASE 
    WHEN sls_sales != sls_quantity * ABS(sls_price) OR sls_sales <= 0 OR sls_sales IS NULL THEN sls_quantity * ABS(sls_price)
    ELSE sls_sales
    END AS sales_amount,
  sls_quantity,
  CASE  
    WHEN sls_price <= 0 OR sls_price IS NULL THEN  sls_sales /NULLIF(sls_quantity,0)
				ELSE sls_price -- Derive price if original value is invalid
			END price
FROM workspace.bronze.crm_sales_details

# Sanity Checks of Target Table

In [0]:
%sql
SELECT * FROM workspace.silver.crm_sales

# View Target Table changes

In [0]:
%sql
DESCRIBE HISTORY workspace.silver.crm_sales